# Machine Learning -- Creating Labels
-----


# Introduction

In this tutorial, we'll discuss how to formulate a policy problem or a social science question in the machine learning framework; how to transform raw data into something that can be fed into a model; how to build, evaluate, compare, and select models; and how to reasonably and accurately interpret model results. You'll also get hands-on experience using the `scikit-learn` package in Python. 

This tutorial is based on chapter "Machine Learning" of [Big Data and Social Science](https://coleridge-initiative.github.io/big-data-and-social-science/).

## Setup

In [ ]:
%pylab inline
import pandas as pd
import sqlite3
from dateutil.parser import parse

In [ ]:
DB = 'ncdoc.db'
conn = sqlite3.connect(DB)
cur = conn.cursor()

# Problem Formulation
---
  
Our Machine Learning Problem
>Of all prisoners released, we would like to predict who is likely to reenter jail within *5* years of the day we make our prediction. For instance, say it is Jan 1, 2009 and we want to identify which 
>prisoners are likely to re-enter jail between now and end of 2013. We can run our predictive model and identify who is most likely at risk. The is an example of a *binary classification* problem. 

Note the outcome window of 5 years is completely arbitrary. You could use a window of 5, 3, 1 years or 1 day. 

In order to predict recidivism, we will be using data from the `inmate` and `sentences` table to create labels (predictors, or independent variables, or $X$ variables) and features (dependent variables, or $Y$ variables). 

We need to munge our data into **labels** (1_Machine_Learning_Labels.ipynb) and **features** (2_Machine_Learning_Features.ipynb) before we can train and evaluate **machine learning models** (3_Machine_Learning_Models.ipynb).

# Creating Labels (Outcomes)
---

First, we create a table `release_dates_2000_2008`, which is based on the `sentences` table. We take all of the records for `inmate_doc_number` and `actual_sentence_end_date` between 2000 and 2008.

In [ ]:
sql_string = "drop table if exists release_dates_2000_2008;"
cur.execute(sql_string)

sql_string = "create temp table release_dates_2000_2008 as "
sql_string += "select inmate_doc_number, actual_sentence_end_date, sentence_begin_date_for_max "
sql_string += "from sentences "
sql_string += "where actual_sentence_end_date >= '2000-01-01' and actual_sentence_end_date <= '2008-12-31' "
sql_string += ";"
cur.execute(sql_string)

Next we create a table `last_exit_2000_2008`, which takes the *maximum* (most recent) `actual_sentence_end_date` for every `inmate_doc_number`. This table will only have one entry per `inmate_doc_number`, so for any given `inmate_doc_number`, or individual, we know their *most recent* release year.

In [ ]:
sql_string = "drop table if exists last_exit_2000_2008;"
cur.execute(sql_string)

sql_string = "create temp table last_exit_2000_2008 as "
sql_string += "select inmate_doc_number, max(actual_sentence_end_date) actual_sentence_end_date "
sql_string += "from release_dates_2000_2008 "
sql_string += "group by inmate_doc_number "
sql_string += ";"
cur.execute(sql_string)

We then find everyone admitted into prison between 2009 and 2013 and create table `admit_2009_2013`.

In [ ]:
sql_string = "drop table if exists admit_2009_2013;"
cur.execute(sql_string)

sql_string = "create temp table admit_2009_2013 as "
sql_string += "select inmate_doc_number, sentence_begin_date_for_max "
sql_string += "from sentences "
sql_string += "where sentence_begin_date_for_max >= '2009-01-01' and sentence_begin_date_for_max <= '2013-12-31' and inmate_sentence_component = 1 "
sql_string += ";"
cur.execute(sql_string)

Next, we do a `left join`  on the `last_exit_2000_2008` (left) table and the `admit_2009_2013` (right) table on the `inmate_doc_number` field. The resulting table will keep all the entries from the left table (most recent releases between 2000 and 2008) and add their admits between 2009 and 2013. Now we can create a label: 0 indicates *no recidivism*, 1 indicates *recidivism*, i.e. that person did return to jail between 2009 and 2013. 

In [ ]:
sql_string = "drop table if exists recidivism_2009_2013;"
cur.execute(sql_string)

sql_string = "create temp table recidivism_2009_2013 as "
sql_string += "select r.inmate_doc_number, r.actual_sentence_end_date, a.sentence_begin_date_for_max, "
sql_string += "case when a.sentence_begin_date_for_max is null then 0 else 1 end recidivism "
sql_string += "from last_exit_2000_2008 r "
sql_string += "left join admit_2009_2013 a on r.inmate_doc_number = a.inmate_doc_number "
sql_string += ";"
cur.execute(sql_string)

We then remove any potential duplicates and create the final label table `recidivism_labels_2009_2013`. 

In [ ]:
sql_string = "drop table if exists recidivism_labels_2009_2013;"
cur.execute(sql_string)

sql_string = "create table public.recidivism_labels_2009_2013 as "
sql_string += "select distinct inmate_doc_number, recidivism "
sql_string += "from recidivism_2009_2013 "
sql_string += ";"
cur.execute(sql_string)

Finally, we load the label table into `label_2009_2013` and inspect the first observations.

In [ ]:
sql_string = "SELECT *"
sql_string += "FROM recidivism_labels_2009_2013 "
sql_string += ";"

label_2009_2013 = pd.read_sql(sql_string, con = conn)
label_2009_2013.head(5)

Following the machine learning pipeline, we will need a second label table for creating a test set later on. To facilitate the label generation process, we define a function called `create_labels` that automates all steps that are needed to create the label table. In essence, it runs all previous steps for a given prediction start date and prediction end date.

In [ ]:
def create_labels(prediction_start, prediction_end, conn):
    """
    Generate a list of labels and return the table as a dataframe.
    
    Parameters
    ----------
    prediction_start
    prediction_end
    conn: obj
        
    Returns
    -------
    df_labels: DataFrame
    """
    start_y = prediction_start
    end_y = prediction_end
    start_y_year = parse(start_y, fuzzy = True).year
    end_y_year = parse(end_y, fuzzy = True).year
    
    sql_script="""

drop table if exists release_dates_2000_{start_y_year};
create temp table release_dates_2000_{start_y_year} as
select inmate_doc_number, actual_sentence_end_date, sentence_begin_date_for_max
from sentences
where actual_sentence_end_date >= '2000-01-01' and actual_sentence_end_date <= '{start_y}';
commit;

drop table if exists last_exit_2000_{start_y_year};
create temp table last_exit_2000_{start_y_year} as
select inmate_doc_number, max(actual_sentence_end_date) actual_sentence_end_date
from release_dates_2000_{start_y_year}
group by inmate_doc_number;
commit;

drop table if exists admit_{start_y_year}_{end_y_year};
create temp table admit_{start_y_year}_{end_y_year} as
select inmate_doc_number, sentence_begin_date_for_max
from sentences
where sentence_begin_date_for_max >= '{start_y}' and sentence_begin_date_for_max <= '{end_y}' and inmate_sentence_component = 1;
commit;

drop table if exists recidivism_{start_y_year}_{end_y_year};
create temp table recidivism_{start_y_year}_{end_y_year} as
select r.inmate_doc_number, r.actual_sentence_end_date, a.sentence_begin_date_for_max,
case when a.sentence_begin_date_for_max is null then 0 else 1 end recidivism
from last_exit_2000_{start_y_year} r
left join admit_{start_y_year}_{end_y_year} a on r.inmate_doc_number = a.inmate_doc_number;
commit;

drop table if exists recidivism_labels_{start_y_year}_{end_y_year};
create table recidivism_labels_{start_y_year}_{end_y_year} as
select distinct inmate_doc_number, recidivism
from recidivism_{start_y_year}_{end_y_year};
commit; 

    """.format(start_y = start_y,
               end_y = end_y,
               start_y_year = start_y_year,
               end_y_year = end_y_year)
    
    cur.execute(sql_script)
    df_label = pd.read_sql('select * from recidivism_labels_{start_y_year}_{end_y_year}'.format(
                                                                                    start_y_year = start_y_year,
                                                                                    end_y_year = end_y_year), conn)    
    return df_label

The `create_labels` function takes a `prediction_start` date and a `prediction_end` date as arguments. Our second label table covers recidivism between 2014 and 2018, based on all releases between 2000 and 2013.

In [ ]:
label_2014_2018 = create_labels('2014-01-01', '2018-12-31', conn)
label_2014_2018.head(5)

In [ ]:
cur.close()
conn.close()